generate conf

In [1]:
multstr = "4 5 16"
rtnamearr = ["ecmp","su2","su3","32disjoint"]
rt1arr = ["ecmp","su","su","kdisjoint"]
rt2arr = [0,2,3,32]
pwnamearr = ["equal","thisweight"]
sseedarr = range(1,4)
trafficfile = "flowfiles/c2s_64_4_4_5_16_0"

with open(f"/home/annzhou/DRing/src/emp/datacentre/experiments/linkfailure2/whatrouting.conf",'w') as f:
    rtname = "ecmp"
    rt1 = "ecmp"
    rt2 = 0
    pwname = "equal"
    pwfile = f"qvarfiles/qvar_leafspine_0_0_{rtname}_64"
    make_leafspine = "MAKE"
    for sseed in sseedarr:
        f.write(f"./run.sh LEAFSPINE {make_leafspine} 80 3072 64 leafspine_nf NEW_FILE null {multstr} 0 0 0 0 0 0 {rt1} {rt2} {trafficfile} {sseed} netpathfiles/netpath_{rtname}_leafspine.txt {pwfile} 64 50 150 200 0 0 > m_whatrouting_leafspine_{rtname}_nf_{pwname}_{sseed}.log\n")
        make_leafspine = "NOMAKE"

    make_dring = "MAKE"
    for irtname,rtname in enumerate(rtnamearr):
        rt1 = rt1arr[irtname]
        rt2 = rt2arr[irtname]
        for pwname in pwnamearr:
            for sseed in sseedarr:
                if pwname == "equal":
                    pwfile = f"qvarfiles/qvar_dring_0_0_{rtname}_64"
                    f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_nf NEW_FILE graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 0 0 0 0 0 0 {rt1} {rt2} {trafficfile} {sseed} netpathfiles/netpath_{rtname}_dring.txt {pwfile} 64 50 150 200 0 0 > m_whatrouting_dring_{rtname}_nf_{pwname}_{sseed}.log\n")
                    make_dring = "NOMAKE"
                else:
                    pwfile = f"qvarfiles/qvar_dring_0_0_{rtname}_c2s_64_4_0_4_5_16_2_0_64"
                    f.write(f"./run.sh RRG {make_dring} 80 2988 64 dring_nf NEW_FILE graphfiles/ring_supergraph/double_ring/instance1_80_64.edgelist {multstr} 0 0 0 0 0 0 {rt1} {rt2} {trafficfile} {sseed} netpathfiles/netpath_{rtname}_dring.txt {pwfile} 64 50 150 200 0 0 > m_whatrouting_dring_{rtname}_nf_{pwname}_{sseed}.log\n")
                    make_dring = "NOMAKE"

generate sumfile & picklefile

In [ ]:
import pickle

homedir = "/home/annzhou"
conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/linkfailure2/whatrouting.conf"
sumfile = f"{homedir}/DRing/src/emp/datacentre/experiments/linkfailure2/whatrouting.txt"
datadict = dict()
with open(conffile,'r') as f:
    with open(sumfile,'a') as sumf:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            logfile = tokens[31]
            sizelist = list()
            durationlist = list()
            startlist = list()
            with open(f"{homedir}/DRing/src/emp/datacentre/{logfile}",'r') as logf:
                print(logfile)
                loglines = logf.readlines()
                for logline in loglines:
                    logtokens = logline.split()
                    if logtokens[0] == "FCT":
                        durationlist.append(float(logtokens[2]))
                        sizelist.append(int(logtokens[1]))
                        startlist.append(float(logtokens[3]))
            datadict[logfile] = [durationlist,sizelist,startlist]
            durationlist.sort()
            sumf.write(f"{logfile}\t{sum(durationlist)/len(durationlist)}\t{durationlist[int(len(durationlist)*0.5)]}\t{durationlist[int(len(durationlist)*0.99)]}\t{durationlist[int(len(durationlist)*0.9999)]}\n")

    with open(f'/home/annzhou/DRing/src/emp/datacentre/experiments/linkfailure2/whatrouting.pickle', 'wb') as handle:
        pickle.dump(datadict, handle, protocol=pickle.HIGHEST_PROTOCOL)

delete logfiles, run, _log

In [ ]:
import os

homedir = "/home/annzhou"
conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/linkfailure2/whatrouting.conf"
with open(conffile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split()
        logfile = tokens[31]
        os.remove(f"{homedir}/DRing/src/emp/datacentre/{logfile}")

dringdir = f"{homedir}/DRing/src/emp/datacentre"
for filedir in os.listdir(dringdir):
    if os.path.isfile(os.path.join(dringdir, filedir)):
        tokens = filedir.split("_")
        if (tokens[0]=="rrg" or tokens[0]=="leafspine") and tokens[1]=="log":
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")
        if (tokens[-1]=="run") and ((tokens[0]=="leafspine" and tokens[1]=="leafspine") or (tokens[0]=="rrg" and (tokens[1]=="rrg" or tokens[1]=="dring"))):
            print(filedir)
            os.remove(f"{homedir}/DRing/src/emp/datacentre/{filedir}")